# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [8]:
# import libraries

import re
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [12]:
# load data from database

def load_data():
    
    engine = create_engine('sqlite:///Disaster_data.db')
    df = pd.read_sql("SELECT * FROM Disaster_data", engine)

    df = df.head(50)
    category_columns = ['related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 'search_and_rescue', 'security', 'military', 'child_alone', 'water', 'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport', 'buildings', 'electricity', 'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report']
    X = df['message']
    Y = df[category_columns]

    return X,Y

In [13]:
load_data()

(0     Weather update - a cold front from Cuba that c...
 1               Is the Hurricane over or is it not over
 2                       Looking for someone but no name
 3     UN reports Leogane 80-90 destroyed. Only Hospi...
 4     says: west side of Haiti, rest of the country ...
 5                Information about the National Palace-
 6                        Storm at sacred heart of jesus
 7     Please, we need tents and water. We are in Sil...
 8       I would like to receive the messages, thank you
 9     I am in Croix-des-Bouquets. We have health iss...
 10    There's nothing to eat and water, we starving ...
 11    I am in Petionville. I need more information r...
 12    I am in Thomassin number 32, in the area named...
 13    Let's do it together, need food in Delma 75, i...
 14    More information on the 4636 number in order f...
 15    A Comitee in Delmas 19, Rue ( street ) Janvier...
 16    We need food and water in Klecin 12. We are dy...
 17    are you going to call me

In [36]:
# take a look at the actual message
df['message'].iloc[0]

'Weather update - a cold front from Cuba that could pass over Haiti'

### 2. Write a tokenization function to process your text data

In [232]:
def tokenize(text):
    
    from nltk.tokenize import word_tokenize
    from nltk.stem import WordNetLemmatizer
    from nltk.corpus import stopwords

    # Remove punctuation characters
    import re
    text = re.sub(r"[^a-zA-Z0-9]", " ", text) 
    
    # tokenize
    raw_tokens = word_tokenize(text)
    
    # remove stop words
    tokens = [w for w in raw_tokens if w not in stopwords.words("english")]
    
    # lemmatize and normalize
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

In [233]:
tokenize(X.iloc[0])

['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti']

In [234]:
 df.message.apply(lambda x: tokenize(x))
# df.message.apply(lambda x: pd.series(tokenize(x)))

KeyboardInterrupt: 

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [1]:
def build_model():
    
    from sklearn.metrics import confusion_matrix
    from sklearn.model_selection import RandomizedSearchCV
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.multioutput import MultiOutputClassifier
    from sklearn.model_selection import train_test_split
    from sklearn.pipeline import Pipeline, FeatureUnion
    from sklearn.base import BaseEstimator, TransformerMixin
    from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])
    
    parameters = {'clf__estimator__n_estimators': [20, 30, 40],
    'clf__estimator__min_samples_split': [2,5,8]}

    model = RandomizedSearchCV(pipeline, param_distributions=parameters, n_jobs=4, verbose=2)

    return model

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [239]:
model = build_model()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

C:\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:266: UserWarning: The total space of parameters 9 is smaller than n_iter=10. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=4)]: Done  27 out of  27 | elapsed:    6.6s finished
C:\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [240]:
print(y_test)

    related  request  offer  aid_related  medical_help  medical_products  \
16        1        1      0            1             1                 0   
1         1        0      0            1             0                 0   
40        1        0      0            1             1                 0   
49        1        0      0            1             0                 0   
33        1        0      0            0             0                 0   
18        0        0      0            0             0                 0   
30        0        0      0            0             0                 0   
21        1        1      0            1             0                 0   
47        1        1      0            1             0                 0   
25        0        0      0            0             0                 0   

    search_and_rescue  security  military  child_alone  ...  aid_centers  \
16                  0         0         0            0  ...            0   
1          

In [249]:
def display_results(y_test, y_pred):
    
    category_columns = ['related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 'search_and_rescue', 'security', 'military', 'child_alone', 'water', 'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport', 'buildings', 'electricity', 'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report']

    from sklearn.metrics import classification_report
    print(classification_report(y_test, y_pred,target_names = category_columns))

In [250]:
display_results(y_test, y_pred)

                        precision    recall  f1-score   support

               related       0.70      1.00      0.82         7
               request       0.50      0.33      0.40         3
                 offer       0.00      0.00      0.00         0
           aid_related       0.67      1.00      0.80         6
          medical_help       0.00      0.00      0.00         2
      medical_products       0.00      0.00      0.00         0
     search_and_rescue       0.00      0.00      0.00         0
              security       0.00      0.00      0.00         0
              military       0.00      0.00      0.00         2
           child_alone       0.00      0.00      0.00         0
                 water       1.00      0.50      0.67         2
                  food       1.00      0.33      0.50         3
               shelter       0.00      0.00      0.00         0
              clothing       0.00      0.00      0.00         0
                 money       0.00      

Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [ ]:
def save_model(model, model_filepath):
    
    pickle.dump(model, open('model.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

## Test the code

In [14]:
import sys

def load_data(database_filepath):
    
    import re
    import numpy as np
    import pandas as pd
    from sqlalchemy import create_engine
    engine = create_engine(database_filepath)
    df = pd.read_sql("SELECT * FROM Disaster_data", engine)

    df = df.head(20)
    category_columns = ['related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 'search_and_rescue', 'security', 'military', 'child_alone', 'water', 'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport', 'buildings', 'electricity', 'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report']
    X = df['message']
    Y = df[category_columns]

    return X,Y,category_columns

def tokenize(text):
    
    from nltk.tokenize import word_tokenize
    from nltk.stem import WordNetLemmatizer
    from nltk.corpus import stopwords

    # Remove punctuation characters
    import re
    text = re.sub(r"[^a-zA-Z0-9]", " ", text) 
    
    # tokenize
    raw_tokens = word_tokenize(text)
    
    # remove stop words
    tokens = [w for w in raw_tokens if w not in stopwords.words("english")]
    
    # lemmatize and normalize
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens


def build_model():
    
    from sklearn.metrics import confusion_matrix
    from sklearn.model_selection import RandomizedSearchCV
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.multioutput import MultiOutputClassifier
    from sklearn.pipeline import Pipeline, FeatureUnion
    from sklearn.base import BaseEstimator, TransformerMixin
    from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])
    
    parameters = {'clf__estimator__n_estimators': [20, 30, 40],
    'clf__estimator__min_samples_split': [2,5,8]}

    model = RandomizedSearchCV(pipeline, param_distributions=parameters, n_jobs=4, verbose=2)

    return model
    

def evaluate_model(model, X_test, Y_test, category_names):
    
    y_pred = model.predict(X_test)

    category_columns = category_names
    from sklearn.metrics import classification_report
    print(classification_report(y_test, y_pred,target_names = category_columns))
    


In [15]:
X, y, category_names = load_data('sqlite:///Disaster_data.db')
model = build_model()

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2)
model.fit(X_train, y_train)
        
evaluate_model(model, X_test, y_test, category_names)

C:\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:266: UserWarning: The total space of parameters 9 is smaller than n_iter=10. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=4)]: Done  27 out of  27 | elapsed:   14.9s finished
C:\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


                        precision    recall  f1-score   support

               related       0.75      1.00      0.86         3
               request       0.00      0.00      0.00         2
                 offer       0.00      0.00      0.00         0
           aid_related       1.00      0.50      0.67         2
          medical_help       0.00      0.00      0.00         1
      medical_products       0.00      0.00      0.00         0
     search_and_rescue       0.00      0.00      0.00         0
              security       0.00      0.00      0.00         0
              military       0.00      0.00      0.00         0
           child_alone       0.00      0.00      0.00         0
                 water       0.00      0.00      0.00         1
                  food       0.00      0.00      0.00         2
               shelter       0.00      0.00      0.00         0
              clothing       0.00      0.00      0.00         0
                 money       0.00      

C:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


In [16]:
model

RandomizedSearchCV(cv='warn', error_score='raise-deprecating',
                   estimator=Pipeline(memory=None,
                                      steps=[('vect',
                                              CountVectorizer(analyzer='word',
                                                              binary=False,
                                                              decode_error='strict',
                                                              dtype=<class 'numpy.int64'>,
                                                              encoding='utf-8',
                                                              input='content',
                                                              lowercase=True,
                                                              max_df=1.0,
                                                              max_features=None,
                                                              min_df=1,
                                     

In [20]:
import pickle
with open('../models/disaster_model.pkl', 'wb') as f:
    pickle.dump(model, f)

In [27]:
def save_model(model, model_filepath):
    import pickle
    with open(model_filepath +'/disaster_model.pkl', 'wb') as f:
        pickle.dump(model, f)

In [28]:
save_model(model, '../models')